In [ ]:
!pip install scipy

# Análisis Espaciales
Los análisis espaciales hacen uso de las características de los objetos (atributos) y de la relación espacial entre elementos de una misma capa y/o de capas superpuestas, para realizar operaciones y devolvernos un resultado. 

<img src="image/datos_geo.png" width="300">

Estas operaciones pueden producir:
* Otra capa con objetos geográficos, de igual o distinto tipo de geometría que la capa original
* Tablas sin geometrías
* Un único valor

## Transformaciones
En esta categoría de análisis espaciales se engloban aquellas operaciones que dan como resultado elementos distintos a los datos de entrada. 
Los procedimientos más frecuentes realizados son: áreas de influencias (buffer), cálculos de centroides de polígonos, generación de intersecciones de calles, etc.
Veremos algunos a continuación:

### Buffer o áreas de influencia
Los objetos geográficos tienen influencia sobre su entorno, Tobler (1970) dijo *“Todas las cosas están relacionadas entre sí, pero las cosas más próximas en el espacio tienen una relación mayor que las distantes”*. Tobler menciona el parámetro de distancia como un factor importante en los cálculos de influencias. 

In [ ]:
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
from owslib.wfs import WebFeatureService

# URL del servicio WFS que deseas consultar
wfs_url = "http://ide.pergamino.gob.ar:8080/geoserver/wfs" # IGN

# Solicitud GetCapabilities al servicio WFS
wfs = WebFeatureService(wfs_url, version='1.0.0')

# Listado de capas publicadas
# list(wfs.contents)

# Paradas de colectivos
paradas_colectivos = gpd.read_file(wfs.getfeature(typename=['paradas_de_colectivo']))
paradas_colectivos = paradas_colectivos.set_crs('EPSG:4326')
paradas_colectivos = paradas_colectivos.to_crs('EPSG:5347') # Se transforma a Gauss Kruger Faja 5 para trabajar en un sistema proyectado

In [ ]:
import folium

mapa = paradas_colectivos.explore(
    style_kwds=dict(stroke=True, # Habilitar borde
                    weight=1, # Grosor en px del borde
                    color = 'red', # Color de borde
                    fillColor='red', # Color de relleno
                    fillOpacity=0.5),  # Opacidad de relleno
    name="Parada Colectivo",  # Nombre del layer en el control de capas
)

mapa # Mostrar mapa

Realizamos un buffer sobre las paradas de colectivo para conocer su área de influencia

In [ ]:
# Creación del buffer
paradas_colectivos_buffer = paradas_colectivos.buffer(300) # Radio de influencia en unidades del sistema de coordenadas de la capa

# Comprobación del objeto resultado
paradas_colectivos_buffer.head()

# Conversión en un GeoDataFrame
paradas_colectivos_buffer = gpd.GeoDataFrame(geometry=paradas_colectivos_buffer, crs=paradas_colectivos.crs) # Indicamos

In [ ]:
mapa = paradas_colectivos_buffer.explore(
                    style_kwds=dict(stroke=True, # Habilitar borde
                    weight=1, # Grosor en px del borde
                    color = 'red', # Color de borde
                    fillColor='red', # Color de relleno
                    fillOpacity=0.2),  # Opacidad de relleno
    name="Área Influencia",  # Nombre del layer en el control de capas
)

folium.LayerControl().add_to(mapa)  # Agregar control de capas al mapa

mapa # Mostrar mapa

### Disolver
Este algoritmo toma una capa vectorial de polígonos y borrará los límites comunes de los polígonos adyacentes. Se pueden especificar uno o más atributos para disolver características pertenecientes a la misma clase (con el mismo valor para los atributos especificados), o bien todos los objetos espaciales se pueden disolver en uno solo.

Apliquemos este algoritmos sobre el área de influencia.


In [ ]:
# Aplicamos un disolver sin agrupamiento por columna
paradas_colectivos_dissolve = paradas_colectivos_buffer.dissolve()

# Comprobación del objeto resultado
paradas_colectivos_dissolve

In [ ]:
mapa = paradas_colectivos_dissolve.explore(
                    style_kwds=dict(stroke=True, # Habilitar borde
                    weight=1, # Grosor en px del borde
                    color = 'red', # Color de borde
                    fillColor='red', # Color de relleno
                    fillOpacity=0.2),  # Opacidad de relleno
    name="Área Influencia Disuelta",  # Nombre del layer en el control de capas
)

mapa # Mostrar mapa

### Polígonos de Voronoi
Este algoritmo permite dividir el territorio en regiones de tal forma que los elementos superpuestos en ellas están más cerca del punto que le da origen que de ningún otro.

![Polígonos de Voronoi](image/voronoi.jpg)

La creación de estos polígonos se realiza a través de la creación de una línea perpendicular a la recta que une 2 puntos y ubicada a la mitad de la distancia entre ellos.

Este algoritmo es utilizado cuando se quieren aproximar la población afectada a un objeto particular, ya que se podría entender que los elementos dentro de la región están siendo influenciados o alcanzados por el punto que le da origen.

Aplicaremos esta transformación para conocer la parada más cercana de acuerdo a la ubicación.

In [ ]:
from scipy.spatial import Voronoi
from shapely.geometry import Polygon
import pandas as pd

# Extraemos coordenadas X e Y de la geometría porque es la forma que lo requiere scipy.spatial
x_coords = paradas_colectivos.geometry.x
y_coords = paradas_colectivos.geometry.y

# Creamos un DataFrame con las coordenadas
coords_paradas = pd.DataFrame({'X': x_coords, 'Y': y_coords})

# Creamos polígonos de Voronoi
vor = Voronoi(coords_paradas)

# Capturamos los vertices de cada polígono
voronoi_polygons = []
for region in vor.regions:
    if not -1 in region and len(region) > 0:
        vertices = [vor.vertices[i] for i in region]
        voronoi_polygons.append(Polygon(vertices))

# Creamos un GeoDataFrame con los polígonos de Voronoi
poligonos_voronoi = gpd.GeoDataFrame(geometry=voronoi_polygons)
poligonos_voronoi = poligonos_voronoi.set_crs(paradas_colectivos.crs) # Utilizamos la capa que le da origen

# Mostramos el GeoDataFrame resultante
mapa = poligonos_voronoi.explore(
                    style_kwds=dict(stroke=True, # Habilitar borde
                    weight=1, # Grosor en px del borde
                    color = 'red', # Color de borde
                    fillColor='red', # Color de relleno
                    fillOpacity=0.2),  # Opacidad de relleno
    name="Área Influencia Disuelta",  # Nombre del layer en el control de capas
)

mapa # Mostrar mapa

### Cortar
Cortamos los polígonos de Voronoi con el área de influencia para enmarcar nuestro problema al área de estudio con el método `overlay`.

In [ ]:
# Cortamos la capa Voronoi con el área de influencia disuelta
resultado_corte = gpd.overlay(poligonos_voronoi, paradas_colectivos_dissolve, how='intersection')

# Mostramos el GeoDataFrame resultante
mapa = resultado_corte.explore(
                    style_kwds=dict(stroke=True, # Habilitar borde
                    weight=1, # Grosor en px del borde
                    color = 'red', # Color de borde
                    fillColor='red', # Color de relleno
                    fillOpacity=0.2),  # Opacidad de relleno
    name="Área Influencia Disuelta",  # Nombre del layer en el control de capas
)

paradas_colectivos.explore(
    m=mapa,  # pass the map object
    style_kwds=dict(stroke=True, # Habilitar borde
                    weight=1, # Grosor en px del borde
                    color = 'blue', # Color de borde
                    fillColor='blue', # Color de relleno
                    fillOpacity=0.5),  # Opacidad de relleno
    name="Parada Colectivo",  # Nombre del layer en el control de capas
)

folium.LayerControl().add_to(mapa)  # use folium to add layer control

mapa # Mostrar mapa

### Centroides
Este algoritmo crea una capa de puntos que representa el centroide de las geometrías de la capa de entrada. El centroide es el punto ubicado en las coordenadas X e Y promedio de los vértices de la geometría de entrada.

![Clase_6/image/centroides.png](image/centroides.png)

In [ ]:
# Barrios
barrios = gpd.read_file(wfs.getfeature(typename=['barrios']))
barrios = barrios.set_crs('EPSG:5347') # Se comprobó que no tenía CRS asignado, se verificaron las coordenadas y se asigna Gauss-Kruguer Faja 5 para tener un sistema proyectado

# Calculos de centroides
barrios_centroides = barrios.geometry.centroid

# Mostramos el GeoDataFrame resultante
mapa = barrios.explore(
                    style_kwds=dict(stroke=True, # Habilitar borde
                    weight=1, # Grosor en px del borde
                    color = 'red', # Color de borde
                    fillColor='red', # Color de relleno
                    fillOpacity=0.2),  # Opacidad de relleno
    name="Barrios",  # Nombre del layer en el control de capas
)

barrios_centroides.explore(
    m=mapa,  # pass the map object
    style_kwds=dict(stroke=True, # Habilitar borde
                    weight=1, # Grosor en px del borde
                    color = 'black', # Color de borde
                    fillColor='black', # Color de relleno
                    fillOpacity=0.5),  # Opacidad de relleno
    name="Centroide",  # Nombre del layer en el control de capas
)

folium.LayerControl().add_to(mapa)  # use folium to add layer control

mapa # Mostrar mapa

## Combinaciones
Son unas de las operaciones más realizadas dentro de los SIG, comprende la superposición de objetos de distintas capas. La naturaleza de la unión por superposición presenta alternativas como son las uniones 1 a 1, 1 a muchos o muchos a muchos.

El predicado espaciales determina la forma en que nuestros objetos geográficos se relacionan en el espacio. Los principales métodos entre la geometría A y otra B son:

![Clase_6/image/TopologicSpatialRelations2.png](image/TopologicSpatialRelations2.png)

* Equals (A, B): Las geometrías son iguales desde un punto de vista topológico
* Disjoint (A, B): No tienen ningún punto en común, las geometrías son disjuntas
* Intersects (A, B):Tienen por lo menos un punto en común. Es el inverso de Disjoint
* Touches (A, B): Las geometrías tendrán por lo menos un punto en común del contorno, pero no puntos interiores
* Crosses (A, B): Comparten parte, pero no todos los puntos interiores, y la dimensión de la intersección es menor que la dimensión de al menos una de las geometrías
* Contains (A, B): Ningún punto de B está en el exterior de A. Al menos un punto del interior de B está en el interior de A
* Within (A, B): Es el inverso de Contains. Within(B, A) = Contains (A, B)
* Overlaps (A, B): Las geometrías comparten parte pero no todos los puntos y la intersección tiene la misma dimensión que las geometrías.
* Covers (A, B): Ningún punto de B está en el exterior de A. B está contenido en A.
* CoveredBy (A, B): Es el inverso de Covers. CoveredBy(A, B) = Covers(B, A)

Fuente: [https://geotalleres.readthedocs.io/es/latest/postgis-relaciones-espaciales/relaciones_espaciales.html](https://geotalleres.readthedocs.io/es/latest/postgis-relaciones-espaciales/relaciones_espaciales.html)



### Unión 1 a 1
Este algoritmo toma una capa vectorial de entrada y crea una nueva capa que contiene los atributos de la capa original junto a los atributos de los objetos superpuestos. Los atributos adicionales y sus valores se toman de una segunda capa vectorial. Se aplica un criterio espacial para seleccionar los valores de la segunda capa que se agregan a cada entidad de la primera capa.

In [ ]:
# Realizar la unión espacial por superposición (contención)
union_1a1 = resultado_corte.sjoin(paradas_colectivos, how="inner", predicate='contains')

# Mostramos el GeoDataFrame resultante
mapa = union_1a1.explore(
                    style_kwds=dict(stroke=True, # Habilitar borde
                    weight=1, # Grosor en px del borde
                    color = 'red', # Color de borde
                    fillColor='red', # Color de relleno
                    fillOpacity=0.2),  # Opacidad de relleno
    name="Área Influencia Disuelta",  # Nombre del layer en el control de capas
)

mapa # Mostrar mapa

### Unión muchos a 1
Cuando realizamos uniones, existe la posibilidad que muchos elementos de la capa a unir cumplan con el predicado geométrico establecido. En ese caso, la herramienta calcula un resumen estadístico de los atributos de los objetos coincidentes en la segunda capa (por ejemplo, valor máximo, valor medio, etc.).

En nuestro caso contaremos el número de paradas de colectivos por barrio.

In [ ]:
# Mostramos el GeoDataFrame resultante
mapa = barrios.explore(
                    style_kwds=dict(stroke=True, # Habilitar borde
                    weight=1, # Grosor en px del borde
                    color = 'red', # Color de borde
                    fillColor='red', # Color de relleno
                    fillOpacity=0.2),  # Opacidad de relleno
    name="Barrios",  # Nombre del layer en el control de capas
)

paradas_colectivos.explore(
    m=mapa,  # pass the map object
    style_kwds=dict(stroke=True, # Habilitar borde
                    weight=1, # Grosor en px del borde
                    color = 'blue', # Color de borde
                    fillColor='blue', # Color de relleno
                    fillOpacity=0.5),  # Opacidad de relleno
    name="Parada Colectivo",  # Nombre del layer en el control de capas
)

folium.LayerControl().add_to(mapa)  # use folium to add layer control

mapa # Mostrar mapa

In [ ]:
# Realizar la unión espacial por superposición (contención)
union = barrios.sjoin(paradas_colectivos, how='inner', predicate='contains')
union.head()

In [ ]:
# Agrupamos la tabla de unión por el identificador único del barrio
conteo_por_barrio = union.groupby('id').size().reset_index(name='cantidad_paradas') #Size devuelve el numero de filas (Pandas)

# Si quisiera calcular el mediana de un campo numérico
# promedio_por_barrio = union.groupby('id')['columna_numerica'].median().reset_index(name='mediana_barrio')

# Fusionar el recuento con el GeoDataFrame de barrios
barrios = barrios.merge(conteo_por_barrio, on='id', how='left')

In [ ]:
# Mostramos el GeoDataFrame resultante
mapa = barrios.explore(
                    style_kwds=dict(stroke=True, # Habilitar borde
                    weight=1, # Grosor en px del borde
                    color = 'red', # Color de borde
                    fillColor='red', # Color de relleno
                    fillOpacity=0.2),  # Opacidad de relleno
    name="Barrios",  # Nombre del layer en el control de capas
)

paradas_colectivos.explore(
    m=mapa,  # pass the map object
    style_kwds=dict(stroke=True, # Habilitar borde
                    weight=1, # Grosor en px del borde
                    color = 'blue', # Color de borde
                    fillColor='blue', # Color de relleno
                    fillOpacity=0.5),  # Opacidad de relleno
    name="Parada Colectivo",  # Nombre del layer en el control de capas
)

folium.LayerControl().add_to(mapa)  # use folium to add layer control

mapa # Mostrar mapa

In [ ]:
# Completamos los datos faltantes en la columna 'cantidad_paradas' con 0
barrios['cantidad_paradas'].fillna(0, inplace=True)

## Cálculo de distancias
Son de las operaciones mas clásicas en SIG. El resultado de este análisis es la distancia entre los objetos más cercanos de 2 capas.

In [ ]:
# Cursos de agua
cursos_de_agua = gpd.read_file(wfs.getfeature(typename=['cursos_de_agua']))
cursos_de_agua = cursos_de_agua.set_crs('EPSG:5347') # Se comprobó que no tenía CRS asignado, se verificaron las coordenadas y se asigna Gauss-Kruguer Faja 5 para tener un sistema proyectado

from shapely.ops import nearest_points
# Crear una lista para almacenar las distancias al río más cercano
distancias_al_rio = []

# Iterar sobre los barrios y calcular la distancia al río más cercano
for idx, barrio in barrios.iterrows():
    punto_barrio = barrio.geometry.centroid
    punto_mas_cercano = nearest_points(punto_barrio, cursos_de_agua.unary_union)[1]
    distancia = punto_barrio.distance(punto_mas_cercano)
    distancias_al_rio.append(distancia)

# Agregar la lista de distancias al GeoDataFrame de barrios
barrios['distancia_al_rio_mas_cercano'] = distancias_al_rio

In [ ]:
# Mostramos el GeoDataFrame resultante
mapa = barrios.explore(
                    style_kwds=dict(stroke=True, # Habilitar borde
                    weight=1, # Grosor en px del borde
                    color = 'red', # Color de borde
                    fillColor='red', # Color de relleno
                    fillOpacity=0.2),  # Opacidad de relleno
    name="Barrios",  # Nombre del layer en el control de capas
)

barrios_centroides.explore(
    m=mapa,  # pass the map object
    style_kwds=dict(stroke=True, # Habilitar borde
                    weight=1, # Grosor en px del borde
                    color = 'black', # Color de borde
                    fillColor='black', # Color de relleno
                    fillOpacity=0.5),  # Opacidad de relleno
    name="Centroide",  # Nombre del layer en el control de capas
)

cursos_de_agua.explore(
    m=mapa,  # pass the map object
    style_kwds=dict(stroke=True, # Habilitar borde
                    weight=2, # Grosor en px del borde
                    color = 'blue'), # Color de borde
    name="Curso de agua",  # Nombre del layer en el control de capas
)

folium.LayerControl().add_to(mapa)  # use folium to add layer control

mapa # Mostrar mapa